## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
enlace

'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210412.ods'

In [5]:
try:
    datos_row = pd.read_excel(enlace,sheet_name='Comunicación')
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210412.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [6]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas con al menos 1 dosis,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,1283580,168000,564800,2016380,1865200,0.925024,1325036,540164,2021-04-11
1,Aragón,250695,35300,88700,374695,312544,0.834129,210392,102152,2021-04-11
2,Asturias,241925,32700,68000,342625,301677,0.880487,204454,97223,2021-04-11
3,Baleares,140760,18000,78000,236760,218353,0.922255,161442,56911,2021-04-11
4,Canarias,276090,35800,145200,457090,387610,0.847995,272256,115354,2021-04-11
5,Cantabria,115395,15300,38800,169495,140961,0.831653,95615,45346,2021-04-11
6,Castilla y Leon,555465,78300,159900,793665,695126,0.875843,461886,233240,2021-04-11
7,Castilla La Mancha,356715,49600,136400,542715,483319,0.890558,331109,152210,2021-04-11
8,Cataluña,1220930,168400,519100,1908430,1733884,0.908539,1275968,457916,2021-04-11
9,C. Valenciana,711550,96800,337500,1145850,1035842,0.903994,751735,284107,2021-04-08


..hacemos una copia para hacerle las transformaciones necesarias:

In [7]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [8]:
datos_hoy.columns

Index(['Unnamed: 0', 'Dosis entregadas Pfizer (1)',
       'Dosis entregadas Moderna (1)', 'Dosis entregadas AstraZeneca (1)',
       'Total Dosis entregadas (1)', 'Dosis administradas (2)',
       '% sobre entregadas', 'Nº Personas con al menos 1 dosis',
       'Nº Personas vacunadas(pauta completada)',
       'Fecha de la última vacuna registrada (2)'],
      dtype='object')

In [9]:
datos_hoy.columns=['ccaa',
                   'Dosis entregadas Pfizer',
                   'Dosis entregadas Moderna',
                   'Dosis entregadas AstraZeneca',
                   'Dosis entregadas',
                   'Dosis administradas',
                   '% sobre entregadas',
                   'Total 1 vacuna',
                   'Total pauta completada',
                   'Fecha de la ultima vacuna registrada']

In [10]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [11]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [12]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [13]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,1283580,168000,564800,2016380,1865200,0.925024,1325036,540164,11/4/21,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,250695,35300,88700,374695,312544,0.834129,210392,102152,11/4/21,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,241925,32700,68000,342625,301677,0.880487,204454,97223,11/4/21,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,140760,18000,78000,236760,218353,0.922255,161442,56911,11/4/21,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,276090,35800,145200,457090,387610,0.847995,272256,115354,11/4/21,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,115395,15300,38800,169495,140961,0.831653,95615,45346,11/4/21,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,555465,78300,159900,793665,695126,0.875843,461886,233240,11/4/21,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,356715,49600,136400,542715,483319,0.890558,331109,152210,11/4/21,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,1220930,168400,519100,1908430,1733884,0.908539,1275968,457916,11/4/21,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,711550,96800,337500,1145850,1035842,0.903994,751735,284107,8/4/21,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [14]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas','Total 1 vacuna',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [15]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [16]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla',
       'Fuerzas Armadas', 'Totales'], dtype=object)

In [17]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,12/4/21,Andalucía,1283580,168000,564800,2016380,1865200,0.925024,1325036,540164,11/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,12/4/21,Aragón,250695,35300,88700,374695,312544,0.834129,210392,102152,11/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,12/4/21,Asturias,241925,32700,68000,342625,301677,0.880487,204454,97223,11/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,12/4/21,Baleares,140760,18000,78000,236760,218353,0.922255,161442,56911,11/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,12/4/21,Canarias,276090,35800,145200,457090,387610,0.847995,272256,115354,11/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,12/4/21,Cantabria,115395,15300,38800,169495,140961,0.831653,95615,45346,11/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,12/4/21,Castilla y Leon,555465,78300,159900,793665,695126,0.875843,461886,233240,11/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,12/4/21,Castilla La Mancha,356715,49600,136400,542715,483319,0.890558,331109,152210,11/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,12/4/21,Cataluña,1220930,168400,519100,1908430,1733884,0.908539,1275968,457916,11/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,12/4/21,C. Valenciana,711550,96800,337500,1145850,1035842,0.903994,751735,284107,8/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [18]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [19]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [20]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368,9/4/21,País Vasco,384750.0,52900.0,148200.0,585850,478177,0.816211,349334.0,128843.0,NaN,8/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1369,9/4/21,Ceuta,9000.0,1300.0,5800.0,16100,15477,0.961304,11836.0,3641.0,NaN,8/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1370,9/4/21,Melilla,9000.0,1200.0,5800.0,16000,13077,0.817312,10143.0,2934.0,NaN,8/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1371,9/4/21,Fuerzas Armadas,19500.0,0.0,66100.0,85600,65711,0.767652,60415.0,5296.0,NaN,8/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [21]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=False)

In [22]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total 1 vacuna', 'Total pauta completada',
       'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [23]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas','Total 1 vacuna',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [24]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,12/4/21,País Vasco,384750.0,52900.0,148200.0,585850,535817,0.914598,400896.0,134921.0,NaN,11/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
17,12/4/21,Ceuta,9000.0,1300.0,5800.0,16100,15799,0.981304,12157.0,3642.0,NaN,9/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
18,12/4/21,Melilla,9000.0,1200.0,5800.0,16000,13690,0.855625,10259.0,3431.0,NaN,10/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
19,12/4/21,Fuerzas Armadas,19500.0,0.0,66100.0,85600,68476,0.799953,63151.0,5325.0,NaN,11/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [25]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales',
       'Fuerzas Armadas'], dtype=object)

y volvemos a subirlo..

In [26]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

# Institucionalizados

In [27]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_institucionalizados_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Institucionalizados'

In [28]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [29]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210412.ods


In [30]:
datos_row

,Unnamed: 0,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA
0,Andalucía,41651,40929,38976,0.982665,0.935776
1,Aragón,18400,18154,17356,0.986630,0.943261
2,Asturias,11948,11617,11149,0.972297,0.933127
3,Baleares,5496,5122,4916,0.931951,0.894469
4,Canarias,10605,10601,9471,0.999623,0.893069
5,Cantabria,7470,7078,6534,0.947523,0.874699
6,Castilla y Leon,46000,45882,44168,0.997435,0.960174
7,Castilla - La Mancha,25566,24510,22929,0.958695,0.896855
8,Cataluña,75649,74073,63275,0.979167,0.836429
9,C. Valenciana,29500,29486,27116,0.999525,0.919186


In [31]:
datos_hoy = datos_row.copy()

In [32]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [33]:
datos_hoy.columns = ['ccaa', 'Denominador Poblacional CCAA',
       'Persona institucionalizada con al menos 1 dosis (2)',
       'Persona institucionalizada  Pauta Completa (2) ',
       '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
       '% Pautas Completas sobre Denominador Poblacional CCAA', 'date_pub',
       'source_name', 'source']

In [34]:
datos_hoy

,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,date_pub,source_name,source
0,Andalucía,41651,40929,38976,0.982665,0.935776,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,18400,18154,17356,0.986630,0.943261,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,11948,11617,11149,0.972297,0.933127,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,5496,5122,4916,0.931951,0.894469,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,10605,10601,9471,0.999623,0.893069,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,7470,7078,6534,0.947523,0.874699,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,46000,45882,44168,0.997435,0.960174,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla - La Mancha,25566,24510,22929,0.958695,0.896855,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,75649,74073,63275,0.979167,0.836429,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,29500,29486,27116,0.999525,0.919186,12/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [35]:
datos_hoy = datos_hoy[['date_pub','ccaa', 'Denominador Poblacional CCAA',
       'Persona institucionalizada con al menos 1 dosis (2)',
       'Persona institucionalizada  Pauta Completa (2) ',
       '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
       '% Pautas Completas sobre Denominador Poblacional CCAA',
       'source_name', 'source']]

In [36]:
datos_hoy

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,12/4/21,Andalucía,41651,40929,38976,0.982665,0.935776,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,12/4/21,Aragón,18400,18154,17356,0.986630,0.943261,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,12/4/21,Asturias,11948,11617,11149,0.972297,0.933127,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,12/4/21,Baleares,5496,5122,4916,0.931951,0.894469,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,12/4/21,Canarias,10605,10601,9471,0.999623,0.893069,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,12/4/21,Cantabria,7470,7078,6534,0.947523,0.874699,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,12/4/21,Castilla y Leon,46000,45882,44168,0.997435,0.960174,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,12/4/21,Castilla - La Mancha,25566,24510,22929,0.958695,0.896855,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,12/4/21,Cataluña,75649,74073,63275,0.979167,0.836429,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,12/4/21,C. Valenciana,29500,29486,27116,0.999525,0.919186,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [37]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_institucionalizados_20210412'

In [38]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

..y ahora lo merge a los acumulados::

In [39]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [40]:
datos_acumulados

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,31/3/21,Andalucía,41651,40770,38848,0.978848,0.932703,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,17829,17829,17025,1.000000,0.954905,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,11948,11577,11115,0.968949,0.930281,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,5496,5117,4869,0.931041,0.885917,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,10531,10531,9441,1.000000,0.896496,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
155,9/4/21,Navarra,8343,8057,7864,0.965720,0.942587,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
156,9/4/21,País Vasco,23000,22924,20333,0.996696,0.884043,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
157,9/4/21,Ceuta,210,195,189,0.928571,0.900000,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
158,9/4/21,Melilla,315,305,292,0.968254,0.926984,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [41]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [42]:
datos_acumulados

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,31/3/21,Andalucía,41651,40770,38848,0.978848,0.932703,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,17829,17829,17025,1.000000,0.954905,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,11948,11577,11115,0.968949,0.930281,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,5496,5117,4869,0.931041,0.885917,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,10531,10531,9441,1.000000,0.896496,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
175,12/4/21,Navarra,8343,8057,7864,0.965720,0.942587,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
176,12/4/21,País Vasco,23000,22924,20333,0.996696,0.884043,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
177,12/4/21,Ceuta,210,195,189,0.928571,0.900000,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
178,12/4/21,Melilla,315,305,292,0.968254,0.926984,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


y volvemos a subirlo:

In [43]:
nombre_fichero_acumulado

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_institucionalizados_'

In [44]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

In [45]:
nombre_fichero_acumulado_long = '/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_institucionalizados_long'
pd.melt(datos_acumulados,id_vars=['date_pub','ccaa','source_name','source'],value_vars=[
    'Denominador Poblacional CCAA',
    'Persona institucionalizada con al menos 1 dosis (2)',
    'Persona institucionalizada  Pauta Completa (2) ',
    '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
    '% Pautas Completas sobre Denominador Poblacional CCAA']
                                      ).to_csv(nombre_fichero_acumulado_long+'.csv',index=False)

# Etarios 1 dosis

In [46]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_etarios_1dosis_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=3,day=31)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Etarios_con_al_menos_1_dosis'

In [47]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [48]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210412.ods


In [49]:
datos_hoy = datos_row.copy()

In [50]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [51]:
datos_hoy.columns = ['ccaa', 'Personas con al menos 1 dosis >=80 años',
       'Población INE>=80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE', 'date_pub',
       'source_name', 'source']

In [52]:
datos_hoy = datos_hoy [['date_pub','ccaa', 'Personas con al menos 1 dosis >=80 años',
       'Población INE>=80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE', 
       'source_name', 'source']]

In [53]:
datos_hoy

,date_pub,ccaa,Personas con al menos 1 dosis >=80 años,Población INE>=80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,12/4/21,Andalucía,412608,413516,0.997804,192640,650381,0.296196,220211,916147,...,625846,0.045292,231,181921,0.001270,1325036,7062213,0.187623,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,12/4/21,Aragón,96328,98807,0.974911,9430,118955,0.079274,22613,156367,...,89180,0.064431,30,25106,0.001195,210392,1132764,0.185733,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,12/4/21,Asturias,85038,87826,0.968255,29644,109249,0.271343,35287,149652,...,54179,0.055132,33,15563,0.002120,204454,901209,0.226866,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,12/4/21,Baleares,46717,49542,0.942978,13864,80516,0.172189,34296,117231,...,85839,0.042941,33,23300,0.001416,161442,986279,0.163688,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,12/4/21,Canarias,75703,90236,0.838945,29882,154531,0.193372,66500,235092,...,163899,0.035308,39,44352,0.000879,272256,1871033,0.145511,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,12/4/21,Cantabria,38221,41042,0.931266,12466,53066,0.234915,9834,77345,...,34935,0.044511,5,10791,0.000463,95615,501384,0.190702,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,12/4/21,Castilla y Leon,213707,223657,0.955512,32593,245410,0.132810,70711,318575,...,144422,0.050934,93,40684,0.002286,461886,2092873,0.220695,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,12/4/21,Castilla - La Mancha,129295,135194,0.956366,16386,159101,0.102991,59944,218444,...,149518,0.038912,93,42364,0.002195,331109,1719964,0.192509,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,12/4/21,Cataluña,422392,447196,0.944534,119350,635292,0.187866,279798,837584,...,556316,0.063025,309,159864,0.001933,1275968,6531658,0.195351,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,12/4/21,C. Valenciana,276732,276531,1.000727,61257,437862,0.139900,107558,580728,...,352834,0.049122,111,103049,0.001077,751735,4269305,0.176079,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [54]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_1dosis_20210412'

In [55]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora merge con los acumulados:

In [56]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [57]:
#datos_acumulados.columns = datos_hoy.columns

In [58]:
datos_acumulados

,date_pub,ccaa,Personas con al menos 1 dosis >=80 años,Población INE>=80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,375382,413516,0.9077810773948287,21914,650381,0.03369409622974841,60757,916147,...,625846,0.03992036379556632,144,181921,0.0007915523771307326,933984,7062213,0.13225089642580873,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,71256,98807,0.7211634803202203,3553,118955,0.029868437644487413,11417,156367,...,89180,0.06159452792105853,28,25106,0.0011152712499004221,161061,1132764,0.14218407364640825,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,67037,87826,0.763293329993396,4258,109249,0.03897518512755266,15803,149652,...,54179,0.05197585780468447,30,15563,0.0019276489108783653,138822,901209,0.15403973994933473,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,33141,49542,0.6689475596463607,2346,80516,0.029137065924785138,15295,117231,...,85839,0.040972052330525754,31,23300,0.0013304721030042918,115375,986279,0.11698008372884346,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,54220,90236,0.600868832838335,15913,154531,0.10297610188247018,33435,235092,...,163899,0.034124674342125336,27,44352,0.0006087662337662338,199517,1871033,0.1066346772077243,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,9/4/21,País Vasco,120749,157950,0.7644761000316556,4641,213081,0.021780449688146762,75703,280119,...,140323,0.08051424214134532,6,41824,0.0001434583014537108,349334,1898123,0.18404181393934957,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
164,9/4/21,Ceuta,2126,2605,0.8161228406909788,1850,4318,0.42843909217230197,2265,7857,...,7581,0.03693444136657433,0,2175,0.0,11836,66158,0.1789050454971432,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
165,9/4/21,Melilla,1597,2349,0.6798637718177948,2164,3748,0.5773745997865528,1435,7827,...,8536,0.022492970946579195,6,2324,0.0025817555938037868,10143,66178,0.15326845779564205,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
166,9/4/21,Fuerzas Armadas,1,-,-,4,-,-,1687,-,...,-,-,0,-,-,60415,-,-,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [59]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [60]:
datos_acumulados

,date_pub,ccaa,Personas con al menos 1 dosis >=80 años,Población INE>=80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,375382,413516,0.9077810773948287,21914,650381,0.03369409622974841,60757,916147,...,625846,0.03992036379556632,144,181921,0.0007915523771307326,933984,7062213,0.13225089642580873,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,71256,98807,0.7211634803202203,3553,118955,0.029868437644487413,11417,156367,...,89180,0.06159452792105853,28,25106,0.0011152712499004221,161061,1132764,0.14218407364640825,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,67037,87826,0.763293329993396,4258,109249,0.03897518512755266,15803,149652,...,54179,0.05197585780468447,30,15563,0.0019276489108783653,138822,901209,0.15403973994933473,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,33141,49542,0.6689475596463607,2346,80516,0.029137065924785138,15295,117231,...,85839,0.040972052330525754,31,23300,0.0013304721030042918,115375,986279,0.11698008372884346,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,54220,90236,0.600868832838335,15913,154531,0.10297610188247018,33435,235092,...,163899,0.034124674342125336,27,44352,0.0006087662337662338,199517,1871033,0.1066346772077243,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,12/4/21,País Vasco,145883,157950,0.923602,27851,213081,0.130706,76721,280119,...,140323,0.081533,6,41824,0.000143,400896,1898123,0.211207,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
185,12/4/21,Ceuta,2145,2605,0.823417,2146,4318,0.496989,2267,7857,...,7581,0.036934,0,2175,0.000000,12157,66158,0.183757,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
186,12/4/21,Melilla,1607,2349,0.684121,2180,3748,0.581644,1520,7827,...,8536,0.022610,6,2324,0.002582,10259,66178,0.155021,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
187,12/4/21,Fuerzas Armadas,3,-,-,4,-,-,2128,-,...,-,-,0,-,-,63151,-,-,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [61]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

In [62]:
nombre_fichero_acumulado_long = '/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_1dosis_long'
pd.melt(datos_acumulados,id_vars=['date_pub','ccaa','source_name','source'],value_vars=[
    'Personas con al menos 1 dosis >=80 años',
       'Población INE>=80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE']
                                      ).to_csv(nombre_fichero_acumulado_long+'.csv',index=False)

# Etarios con pauta completa

In [63]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_etarios_pauta_completa_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Etarios_con_pauta_completa'

In [64]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [65]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210412.ods


In [66]:
datos_hoy = datos_row.copy()

In [67]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [68]:
datos_hoy.columns = ['ccaa', 'Personas pauta completa >=80 años',
       'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE', 'date_pub',
       'source_name', 'source']

In [69]:
datos_hoy = datos_hoy[['date_pub','ccaa', 'Personas pauta completa >=80 años',
       'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE',
       'source_name', 'source']]

In [70]:
datos_hoy

,date_pub,ccaa,Personas pauta completa >=80 años,Población INE>=80 años,%,Personas pauta completa 70-79 años,Población INE70-79 años,%.1,Personas pauta completa 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas pauta completa,Total Población INE Población a Vacunar (1),% pauta completa sobre Población a Vacunar INE,source_name,source
0,12/4/21,Andalucía,266694,413516,0.644942,14552,650381,0.022375,42633,916147,...,625846,0.019276,97,181921,0.000533,540164,7062213,0.076487,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,12/4/21,Aragón,51967,98807,0.525945,3306,118955,0.027792,9539,156367,...,89180,0.016327,25,25106,0.000996,102152,1132764,0.090179,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,12/4/21,Asturias,46107,87826,0.524981,3729,109249,0.034133,10102,149652,...,54179,0.043541,25,15563,0.001606,97223,901209,0.107881,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,12/4/21,Baleares,23105,49542,0.466372,2226,80516,0.027647,5023,117231,...,85839,0.014655,30,23300,0.001288,56911,986279,0.057703,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,12/4/21,Canarias,37645,90236,0.417184,13795,154531,0.089270,9527,235092,...,163899,0.014668,21,44352,0.000473,115354,1871033,0.061653,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,12/4/21,Cantabria,23155,41042,0.564178,1694,53066,0.031923,3755,77345,...,34935,0.017518,3,10791,0.000278,45346,501384,0.090442,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,12/4/21,Castilla y Leon,128589,223657,0.574938,9756,245410,0.039754,20161,318575,...,144422,0.018259,85,40684,0.002089,233240,2092873,0.111445,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,12/4/21,Castilla - La Mancha,71767,135194,0.530845,5769,159101,0.036260,13760,218444,...,149518,0.023723,85,42364,0.002006,152210,1719964,0.088496,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,12/4/21,Cataluña,168413,447196,0.376598,28417,635292,0.044731,49335,837584,...,556316,0.029359,213,159864,0.001332,457916,6531658,0.070107,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,12/4/21,C. Valenciana,143190,276531,0.517808,10440,437862,0.023843,23516,580728,...,352834,0.014695,100,103049,0.000970,284107,4269305,0.066546,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [71]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_20210412'

In [72]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora los merge con los acumulados

In [73]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [74]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [75]:
#datos_acumulados.columns = datos_hoy.columns

In [76]:
datos_acumulados

,date_pub,ccaa,Personas pauta completa >=80 años,Población INE>=80 años,%,Personas pauta completa 70-79 años,Población INE70-79 años,%.1,Personas pauta completa 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas pauta completa,Total Población INE Población a Vacunar (1),% pauta completa sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,240454,413516,0.5814865688389325,12788,650381,0.019662321008762557,41807,916147,...,625846,0.018677118652192393,83,181921,0.0004562419951517417,507430,7062213,0.0718514154132706,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,40245,98807,0.40730919874097987,3048,118955,0.025623134798873525,9273,156367,...,89180,0.015934065934065933,25,25106,0.0009957779016968055,89278,1132764,0.07881429847699963,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,37353,87826,0.4253068567394621,3490,109249,0.031945372497688765,9709,149652,...,54179,0.04263644585540523,24,15563,0.0015421191287026924,87093,901209,0.09664018002483331,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,12403,49542,0.2503532356384482,2088,80516,0.025932733866560684,4856,117231,...,85839,0.014317501368841669,30,23300,0.0012875536480686696,45509,986279,0.04614211597326923,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,31183,90236,0.34557161221685356,11216,154531,0.07258090609651138,9016,235092,...,163899,0.014331997144582944,19,44352,0.0004283910533910534,104712,1871033,0.055964806606831626,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,12/4/21,País Vasco,60159,157950,0.380874,3633,213081,0.017050,12636,280119,...,140323,0.021451,6,41824,0.000143,134921,1898123,0.071081,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
185,12/4/21,Ceuta,1584,2605,0.608061,373,4318,0.086383,313,7857,...,7581,0.005804,0,2175,0.000000,3642,66158,0.055050,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
186,12/4/21,Melilla,1310,2349,0.557684,472,3748,0.125934,294,7827,...,8536,0.006560,6,2324,0.002582,3431,66178,0.051845,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
187,12/4/21,Fuerzas Armadas,1,-,-,4,-,-,471,-,...,-,-,0,-,-,5325,-,-,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [77]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Personas pauta completa >=80 años',
       'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE', 'source_name',
       'source'],
      dtype='object')

In [78]:
nombre_fichero_acumulado

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_'

In [79]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

In [80]:
nombre_fichero_acumulado_long = '/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_long'
pd.melt(datos_acumulados,id_vars=['date_pub','ccaa','source_name','source'],value_vars=[
    'Personas pauta completa >=80 años',
    'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
    'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
    'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
    'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
    'Población INE25-49 años', '%.4', '18-24 años',
    'Población INE18-24 años', '%.5', '16-17 años',
    'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
    'Total Población INE Población a Vacunar (1)',
    '% pauta completa sobre Población a Vacunar INE']
                                      ).to_csv(nombre_fichero_acumulado_long+'.csv',index=False)